In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [2]:
start_date = datetime.date(2020, 1, 21)
end_date = datetime.date(2020, 2, 8)
day_range = (end_date - start_date).days

In [3]:
#Source: John Hopkins interactive map https://bit.ly/2Sbez6b
nCov_confirmed_data = pd.read_csv(r'C:\Users\David Lau\Desktop\nCov_data_confirmed_20200208.csv')
nCov_death_data = pd.read_csv(r'C:\Users\David Lau\Desktop\nCov_data_death_20200208.csv')

In [4]:
#Some days include more than one set of data. Removing the data in the earlier of the day
remove_column = ["1/24/2020 12:00 AM",
"1/25/2020 12:00 AM",
"1/25/2020 12:00 PM",
"1/26/2020 11:00 AM",
"1/27/2020 9:00 AM",
"1/27/2020 7:00 PM",
"1/28/2020 1:00 PM",
"1/28/2020 6:00 PM",
"1/29/2020 1:30 PM",
"1/29/2020 2:30 PM",
"1/31/2020 2:00 PM",
"2/4/2020 9:40",
"2/5/2020 9:00",
"2/6/2020 9:00",
"2/7/2020 20:13",
"2/8/2020 22:04"]

def RemoveDate (original_file):
    revised_file = original_file
    for n in remove_column:
        revised_file = revised_file.drop(n, axis=1)
    return revised_file

In [5]:
revised_nCov_confirmed_data = RemoveDate(nCov_confirmed_data)
revised_nCov_death_data = RemoveDate(nCov_death_data)

In [6]:
def RenameDate(original_file):
    original_file.rename(
        columns={
            "1/21/2020 10:00 PM": datetime.date(2020, 1, 21),
            "1/22/2020 12:00 PM": datetime.date(2020, 1, 22),
            "1/23/2020 12:00 PM": datetime.date(2020, 1, 23),
            "1/24/2020 12:00 PM": datetime.date(2020, 1, 24),
            "1/25/2020 10:00 PM": datetime.date(2020, 1, 25),
            "1/26/2020 11:00 PM": datetime.date(2020, 1, 26),
            "1/27/2020 8:30 PM": datetime.date(2020, 1, 27),
            "1/28/2020 11:00 PM": datetime.date(2020, 1, 28),
            "1/29/2020 9:00 PM": datetime.date(2020, 1, 29),
            "1/30/2020 11:00 AM": datetime.date(2020, 1, 30),
            "1/31/2020 7:00 PM": datetime.date(2020, 1, 31),
            "2/1/2020 10:00": datetime.date(2020, 2, 1),
            "2/2/2020 21:00": datetime.date(2020, 2, 2),
            "2/3/2020 21:00": datetime.date(2020, 2, 3),
            "2/4/2020 22:00": datetime.date(2020, 2, 4),
            "2/5/2020 23:00": datetime.date(2020, 2, 5),
            "2/6/2020 14:20": datetime.date(2020, 2, 6),
            "2/7/2020 22:50": datetime.date(2020, 2, 7),
            "2/8/2020 23:04": datetime.date(2020,2,8)
        },
        inplace=True
    )
    return original_file

In [7]:
revised_nCov_confirmed_data = RenameDate(revised_nCov_confirmed_data)
revised_nCov_death_data = RenameDate(revised_nCov_death_data)

In [8]:
#Group Macau in Mainland China
revised_nCov_confirmed_data.at[38,"Country/Region"] = "Mainland China"
revised_nCov_death_data.at[38,"Country/Region"] = "Mainland China"

In [9]:
death_rate = revised_nCov_confirmed_data.copy()
death_rate.loc[:,start_date:end_date] = revised_nCov_death_data.loc[:,start_date:end_date].div(revised_nCov_confirmed_data.loc[:,start_date:end_date])

In [10]:
revised_nCov_confirmed_data_China = revised_nCov_confirmed_data[revised_nCov_confirmed_data["Country/Region"] == "Mainland China"]
revised_nCov_death_data_China = revised_nCov_death_data[revised_nCov_death_data["Country/Region"] == "Mainland China"]
revised_nCov_confirmed_data_ExChina = revised_nCov_confirmed_data[revised_nCov_confirmed_data["Country/Region"] != "Mainland China"]
revised_nCov_death_data_ExChina = revised_nCov_death_data[revised_nCov_death_data["Country/Region"] != "Mainland China"]

In [11]:
revised_nCov_confirmed_data_ChinaExHubei = revised_nCov_confirmed_data_China[revised_nCov_confirmed_data_China["Province/State"] != "Hubei"]
revised_nCov_death_data_ChinaExHubei = revised_nCov_death_data_China[revised_nCov_death_data_China["Province/State"] != "Hubei"]
revised_nCov_confirmed_data_Hubei = revised_nCov_confirmed_data_China[revised_nCov_confirmed_data_China["Province/State"] == "Hubei"]
revised_nCov_death_data_Hubei = revised_nCov_death_data_China[revised_nCov_death_data_China["Province/State"] == "Hubei"]

In [12]:
total_confirmed = revised_nCov_confirmed_data.sum(axis=0,)
total_death = revised_nCov_death_data.sum(axis=0)
total_death_rate = total_death / total_confirmed

total_confirmed_China = revised_nCov_confirmed_data_China.sum(axis=0)
total_death_China = revised_nCov_death_data_China.sum(axis=0)
total_confirmed_China = total_confirmed_China.drop(['Province/State'])
total_confirmed_China = total_confirmed_China.drop(['Country/Region'])
total_death_rate_China = total_death_China / total_confirmed_China

total_confirmed_ExChina = revised_nCov_confirmed_data_ExChina.sum(axis=0)
total_death_ExChina = revised_nCov_death_data_ExChina.sum(axis=0)
total_death_rate_ExChina = total_death_ExChina / total_confirmed_ExChina

total_confirmed_ChinaExHubei = revised_nCov_confirmed_data_ChinaExHubei.sum(axis=0)
total_death_ChinaExHubei = revised_nCov_death_data_ChinaExHubei.sum(axis=0)
total_confirmed_ChinaExHubei = total_confirmed_ChinaExHubei.drop(['Province/State'])
total_confirmed_ChinaExHubei = total_confirmed_ChinaExHubei.drop(['Country/Region'])
total_death_rate_ChinaExHubei = total_death_ChinaExHubei / total_confirmed_ChinaExHubei

revised_nCov_confirmed_data_Hubei_temp = revised_nCov_confirmed_data_Hubei.loc[:,start_date:end_date]
revised_nCov_death_data_Hubei_temp = revised_nCov_death_data_Hubei.loc[:,start_date:end_date]

total_confirmed_Hubei = revised_nCov_confirmed_data_Hubei_temp.sum(axis=0)
total_death_Hubei = revised_nCov_death_data_Hubei_temp.sum(axis=0)
total_death_rate_Hubei = total_death_Hubei / total_confirmed_Hubei

In [13]:
print('Hubei death rate is {r:1.4f}'.format(r=total_death_rate_Hubei[end_date]))
print('Outside Hubei death rate is {r:1.4f}'.format(r=total_death_rate_ChinaExHubei[end_date]))
print('Hubei death rate is {r:1.2f} times higher than outside Hubei'.format(r=total_death_rate_Hubei[end_date] / total_death_rate_ChinaExHubei[end_date]))

Hubei death rate is 0.0288
Outside Hubei death rate is 0.0031
Hubei death rate is 9.38 times higher than outside Hubei


In [14]:
result = pd.concat([total_death_rate,total_death_rate_Hubei,total_death_rate_ChinaExHubei], axis=1, sort=False)

In [15]:
def plotGraph(ChinaList, HubeiList, ExHubeiList):
    fig, ax = plt.subplots()
    ChinaList.plot(label='China Overall')
    HubeiList.plot(label='Hubei')
    ExHubeiList.plot(label='Ex-Hubei')
    plt.legend(loc='upper right', borderaxespad=0.)
    loc = mticker.MultipleLocator(base=4.0)
    ax.xaxis.set_major_locator(loc)
    plt.show()

In [16]:
def TotalToNew(total_list):
    new_list = [0]
    for i in total_list[start_date:end_date - timedelta(days=1)]:
        new_list.append(i)
    return total_list-new_list

In [17]:
new_confirmed = TotalToNew(total_confirmed)
new_death = TotalToNew(total_death)

new_confirmed_China = TotalToNew(total_confirmed_China)
new_death_China = TotalToNew(total_death_China)

new_confirmed_Hubei = TotalToNew(total_confirmed_Hubei)
new_death_Hubei = TotalToNew(total_death_Hubei)

new_confirmed_ChinaExHubei = TotalToNew(total_confirmed_ChinaExHubei)
new_death_ChinaExHubei = TotalToNew(total_death_ChinaExHubei)

new_confirmed_ExChina = TotalToNew(total_confirmed_ExChina)
new_death_ExChina = TotalToNew(total_death_ExChina)

In [18]:
def movingAvg(original):
    new = original.copy()
    l = len(original)
    new[0] = (original[0]+original[1])/2
    for i in range(l-2):
        new[i] = (original[i]+original[i+1]+original[i+2])/3
    new[l-1] = (original[l-2]+original[l-1])/2
    return new

In [19]:
new_confirmed_avg = movingAvg(new_confirmed)
new_death_avg = movingAvg(new_death)

new_confirmed_China_avg = movingAvg(new_confirmed_China)
new_death_China_avg = movingAvg(new_death_China)

new_confirmed_Hubei_avg = movingAvg(new_confirmed_Hubei)
new_death_Hubei_avg = movingAvg(new_death_Hubei)

new_confirmed_ChinaExHubei_avg = movingAvg(new_confirmed_ChinaExHubei)
new_death_ChinaExHubei_avg = movingAvg(new_death_ChinaExHubei)

new_confirmed_ExChina_avg = movingAvg(new_confirmed_ExChina)
new_death_ExChina_avg = movingAvg(new_death_ExChina)

In [20]:
print('Death rate in:')
print('China overall: {r:1.4f}'.format(r=total_death_rate_China[end_date]))
print('Hubei: {r:1.4f}'.format(r=total_death_rate_Hubei[end_date]))
print('Outside Hubei: {r:1.4f}'.format(r=total_death_rate_ChinaExHubei[end_date]))
print('Outside China: {r:1.4f}'.format(r=total_death_rate_ExChina[end_date]))

Death rate in:
China overall: 0.0218
Hubei: 0.0288
Outside Hubei: 0.0031
Outside China: 0.0059


In [21]:
def calc_delayed_death_rate(confirmed, death, days_delayed):
    rate = confirmed.copy()
    l = len(confirmed)
    for i in range(days_delayed):
        rate[i] = 0
    for i in range(days_delayed,l):
        rate[i] = death[i] / confirmed[i-days_delayed]
    return rate

In [22]:
print('Hubei death rate assuming different days of delay from confirmed to death:')
for i in range(8):
    cummulated_delayed_death_rate = 0
    delayed_days = i
    cummulated_delayed_death_rate = total_death_Hubei[end_date] / total_confirmed_Hubei[end_date - timedelta(days=delayed_days)]
    print('{q} days: {r:1.4f}'.format(r=cummulated_delayed_death_rate,q=i))

Hubei death rate assuming different days of delay from confirmed to death:
0 days: 0.0288
1 days: 0.0313
2 days: 0.0397
3 days: 0.0397
4 days: 0.0468
5 days: 0.0577
6 days: 0.0698
7 days: 0.1090


In [23]:
def CAGR(list, start, end):
    diff = (end-start).days
    return (list[end]/list[start])**(1/diff)-1

In [24]:
city_week_CAGR = pd.DataFrame(columns=['Province/State','Country/Region','firstweek',"secondweek","thirdweek"])
for i in range(len(revised_nCov_confirmed_data)):
    firstweek = (revised_nCov_confirmed_data.at[i,datetime.date(2020,1,25)]/revised_nCov_confirmed_data.at[i,datetime.date(2020,1,21)])**(1/4)-1
    secondweek = (revised_nCov_confirmed_data.at[i,datetime.date(2020,2,1)]/revised_nCov_confirmed_data.at[i,datetime.date(2020,1,26)])**(1/6)-1    
    thirdweek = (revised_nCov_confirmed_data.at[i,datetime.date(2020,2,8)]/revised_nCov_confirmed_data.at[i,datetime.date(2020,2,2)])**(1/6)-1
    city = revised_nCov_confirmed_data.at[i,'Province/State']
    country = revised_nCov_confirmed_data.at[i,'Country/Region']
    city_week_CAGR=city_week_CAGR.append(
        {'Province/State':city,
         "Country/Region":country,
         'firstweek':firstweek,
         'secondweek':secondweek,
         'thirdweek':thirdweek},
        ignore_index=True)

In [25]:
print(city_week_CAGR[(city_week_CAGR['thirdweek']>city_week_CAGR['secondweek'])])
print('Cities that show non-bell-curve infection rate growth:')
print(city_week_CAGR[(city_week_CAGR['thirdweek']>city_week_CAGR['secondweek'])]['Province/State'])

   Province/State  Country/Region  firstweek  secondweek  thirdweek
38          Macau  Mainland China        NaN    0.026025   0.037891
39      Hong Kong       Hong Kong        NaN    0.084282   0.096008
Cities that show non-bell-curve infection rate growth:
38        Macau
39    Hong Kong
Name: Province/State, dtype: object


In [26]:
city_week_CAGR_indexed = city_week_CAGR.set_index("Province/State", drop = False)

print('Hubei death rate during the following period:')
print('1st week (21-25/1): {r:1.4f}'.format(r=city_week_CAGR_indexed.loc['Hubei','firstweek']))
print('2nd week (26/1-1/2): {r:1.4f}'.format(r=city_week_CAGR_indexed.loc['Hubei','secondweek']))
print('3rd week (2-8/2): {r:1.4f}'.format(r=city_week_CAGR_indexed.loc['Hubei','thirdweek']))

Hubei death rate during the following period:
1st week (21-25/1): 0.4050
2nd week (26/1-1/2): 0.3088
3rd week (2-8/2): 0.1591


In [47]:
confirmed_rebase = pd.DataFrame(columns=['Province/State','Country/Region','targetdate','fiftydate','hundreddate'])
for i in range(len(revised_nCov_confirmed_data)):
    city = revised_nCov_confirmed_data.at[i,'Province/State']
    country = revised_nCov_confirmed_data.at[i,'Country/Region']

    date=datetime.date(2020,1,20)
    k=1
    for count in revised_nCov_confirmed_data.loc[i,start_date:end_date]:
        if count>=26:
            break
        k=k+1
    date = date+timedelta(days=k)
    
    fiftydate=datetime.date(2020,1,20)
    m=1
    for count in revised_nCov_confirmed_data.loc[i,start_date:end_date]:
        if count>=50:
            break
        m=m+1
    fiftydate = fiftydate+timedelta(days=m)    

    hundreddate=datetime.date(2020,1,20)
    n=1
    for count in revised_nCov_confirmed_data.loc[i,start_date:end_date]:
        if count>=100:
            break
        n=n+1
    hundreddate = hundreddate+timedelta(days=n)    
    
    confirmed_rebase=confirmed_rebase.append(
        {'Province/State':city,
         "Country/Region":country,
         'targetdate':date,
         'fiftydate':fiftydate,
         'hundreddate':hundreddate},
        ignore_index=True)

In [48]:
confirmed_rebase_cut = confirmed_rebase[
    (confirmed_rebase['targetdate']!=(end_date + timedelta(days=1))) &
    (confirmed_rebase['Country/Region']=='Mainland China') &
    (confirmed_rebase['Province/State']!='Hubei')]

In [49]:
confirmed_rebase_cut_indexed = confirmed_rebase_cut.set_index("Province/State", drop = False)
revised_nCov_confirmed_data_indexed = revised_nCov_confirmed_data.set_index("Province/State", drop = False)

In [50]:
confirmed_rebase_cut_indexed_temp = confirmed_rebase_cut_indexed.copy()

avg_confirmed_by_day = []
for i in range(day_range):
    l = []
    for index,row in confirmed_rebase_cut_indexed.iterrows():
        ref_date = row['targetdate'] + timedelta(days=i)
        try:
            dayzero = revised_nCov_confirmed_data_indexed.loc[row['Province/State'],ref_date]
        except:
            dayzero = np.nan
        l.append(dayzero)
    l = pd.Series(l)
    avg_confirmed_by_day.append(l.sum()/(len(l)-l.isna().sum()))

print('HK confirmed cases forecast:')
for i in range(len(avg_confirmed_by_day)):
    print('Day {x} {y}: {z:1.0f}'.format(
        x=i,
        y=datetime.date(2020,2,9)+timedelta(days=i),
        z=avg_confirmed_by_day[i]))


HK confirmed cases forecast:
Day 0 2020-02-09: 36
Day 1 2020-02-10: 50
Day 2 2020-02-11: 66
Day 3 2020-02-12: 85
Day 4 2020-02-13: 108
Day 5 2020-02-14: 122
Day 6 2020-02-15: 150
Day 7 2020-02-16: 183
Day 8 2020-02-17: 232
Day 9 2020-02-18: 265
Day 10 2020-02-19: 305
Day 11 2020-02-20: 361
Day 12 2020-02-21: 392
Day 13 2020-02-22: 545
Day 14 2020-02-23: 600
Day 15 2020-02-24: 698
Day 16 2020-02-25: 1075
Day 17 2020-02-26: 1120


In [51]:
confirmed_rebase_cut_indexed

,Province/State,Country/Region,targetdate,fiftydate,hundreddate
Province/State,,,,,
Anhui,Anhui,Mainland China,2020-01-25,2020-01-25,2020-01-27
Beijing,Beijing,Mainland China,2020-01-24,2020-01-25,2020-01-29
Chongqing,Chongqing,Mainland China,2020-01-24,2020-01-25,2020-01-26
Fujian,Fujian,Mainland China,2020-01-26,2020-01-27,2020-01-29
Gansu,Gansu,Mainland China,2020-01-29,2020-02-02,2020-02-09
Guangdong,Guangdong,Mainland China,2020-01-22,2020-01-24,2020-01-26
Guangxi,Guangxi,Mainland China,2020-01-25,2020-01-27,2020-02-01
Guizhou,Guizhou,Mainland China,2020-01-31,2020-02-03,2020-02-09
Hainan,Hainan,Mainland China,2020-01-27,2020-01-31,2020-02-05


In [60]:
#26 cases is the HK no. of case now
print('No. of days for each city to reach from >=26 cases to 50/100 cases:')
for index,row in confirmed_rebase_cut_indexed.iterrows():
    if row['fiftydate'] == (end_date + timedelta(days=1)):
        row['fiftydate'] = np.nan
    if row['hundreddate'] == (end_date + timedelta(days=1)):
        row['hundreddate'] = np.nan
    
    try:
        fifty_days = (row['fiftydate']-row['targetdate']).days
    except:
        fifty_days = np.nan

    try:
        hundred_days = (row['hundreddate']-row['targetdate']).days
    except:
        hundred_days = np.nan

    print('{x}: {y}/{z} days'.format(x=index, y=fifty_days, z=hundred_days))


No. of days for each city to reach from >=26 cases to 50/100 cases:
Anhui: 0/2 days
Beijing: 1/5 days
Chongqing: 1/2 days
Fujian: 1/3 days
Gansu: 4/nan days
Guangdong: 2/4 days
Guangxi: 2/7 days
Guizhou: 3/nan days
Hainan: 4/9 days
Hebei: 2/6 days
Heilongjiang: 4/7 days
Henan: 0/1 days
Hunan: 0/1 days
Inner Mongolia: 5/nan days
Jiangsu: 2/4 days
Jiangxi: 2/3 days
Jilin: 2/nan days
Liaoning: 4/12 days
Ningxia: nan/nan days
Shaanxi: 1/5 days
Shandong: 1/3 days
Shanghai: 1/4 days
Shanxi: 5/10 days
Sichuan: 1/3 days
Tianjin: 5/nan days
Xinjiang: nan/nan days
Yunnan: 2/6 days
Zhejiang: 2/2 days
